In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from obspy import read
from obspy.core import UTCDateTime

from pysep import Pysep

from lib_seis import station_map_and_table
from lib_seis import sumatra_waveform_screening
from lib_seis import wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# download raw LHZ data

download  = dict( client                       = 'IRIS',
                  origin_time                  = UTCDateTime('2004,12,26,00,58,53'),
                  event_latitude               = 3.09,
                  event_longitude              = 94.26,
                  event_depth_km               = 30,
                  event_magnitude              = 9.1,
                  networks                     = 'G,II,IU',
                  stations                     = 'ALE,QSPA,NNA,BBSR,SUR,PAYG',
                  locations                    = '00',
                  channels                     = 'LHZ',
                  remove_response              = False,
                  remove_clipped               = False,
                  remove_insufficient_length   = False,
                  detrend                      = False,
                  demean                       = False,
                  pre_filt                     = None,
                  fill_data_gaps               = 0,
                  gap_fraction                 = 1,
                  resample_freq                = 1,
                  seconds_before_ref           =  0.5 * 24 * 60 * 60,
                  seconds_after_ref            = 10.0 * 24 * 60 * 60,
                  taup_model                   = None,
                  log_level                    = 'DEBUG',
                  write_files                  = 'sac',
                  plot_files                   = 'map',
                  output_dir                   = 'datawf',
                  overwrite_event_tag          = 'sumatra_testing_raw')

try:
    ps = Pysep(**download)
    ps.run()
except :
    print('data directory already exists')

In [ ]:
# download LHZ data with instrument response removed

download  = dict( client                       = 'IRIS',
                  origin_time                  = UTCDateTime("2004,12,26,00,58,53"),
                  event_latitude               = 3.09,
                  event_longitude              = 94.26,
                  event_depth_km               = 30,
                  event_magnitude              = 9.1,
                  networks                     = 'G,II,IU',
                  stations                     = 'ALE,QSPA,NNA,BBSR,SUR,PAYG',
                  locations                    = '00',
                  channels                     = 'LHZ',
                  remove_response              = True,
                  output_unit                  = 'ACC',
                  water_level                  = None,
                  remove_clipped               = True,
                  remove_insufficient_length   = True,
                  detrend                      = True,
                  demean                       = True,
                  pre_filt                     = None,
                  fill_data_gaps               = 0,
                  gap_fraction                 = 0.03,
                  resample_freq                = 1,
                  seconds_before_ref           =  0.5 * 24 * 60 * 60,
                  seconds_after_ref            = 10.0 * 24 * 60 * 60,
                  taup_model                   = None,
                  log_level                    = 'DEBUG',
                  write_files                  = 'sac',
                  plot_files                   = 'map',
                  output_dir                   = 'datawf',
                  overwrite_event_tag          = 'sumatra_testing_acc_water_level')

try:
    ps = Pysep(**download)
    ps.run()
except :
    print('data directory already exists')

In [ ]:
# read downloaded raw LHZ data

sac_file = f'./datawf/sumatra_testing_raw/SAC/*.sac'
st       = read(sac_file, header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# read downloaded accel LHZ data

sac_file = f'./datawf/sumatra_testing_acc_water_level/SAC/*.sac'
st2       = read(sac_file, header=None)

print('Details of downloaded waveforms - \n')
print(st2.__str__(extended=True))
print()

In [ ]:
# storing station information corresponding to retained waveforms
station_map_and_table(st, event_lat=download['event_latitude'], event_lon=download['event_longitude'])

In [ ]:
# computing and plotting the amplitude spectra of the retained waveforms

if True:
    Nyquist_frequency = download['resample_freq']/2
    for i in range(len(st)):

        tr = st[i].copy()  
        tr.detrend('constant')
        tr.detrend('linear')
        tr.taper(max_percentage=0.2, type="cosine")

        fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)

        plt.figure()
        plt.plot(fft_freq*1E3,fft_amp)
        plt.xlim(0.2,1)
        #plt.ylim(0,40)
        xmask = np.logical_and(fft_freq*1e3 > .2, fft_freq*1e3 < 1)
        plt.ylim(0,np.max(fft_amp[xmask])*1.1)
        plt.title(f'{tr.id}')
        plt.xlabel(f'Frequency (mHz)')
        plt.ylabel(f'Amplitude (counts s)')
        plt.show()        